In [9]:
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix
import pickle

In [3]:
data = pd.read_csv('./../data/round.csv')

In [4]:
data.shape

(254628, 12)

In [5]:
data.head()

,Tournament_Year,Tournament_#,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Tee_Time,Round_Score,End_of_Event_Pos._(text),Finishing_Pos,Finishing_Pct
0,2003,10,16,656,1320,"Estes, Bob",1,11:50,66,T6,7.5,0.197917
1,2003,10,16,656,1361,"Forsman, Dan",1,11:00,69,T21,22.0,0.600694
2,2003,10,16,656,1761,"Mattiace, Len",1,13:00,75,24,24.0,0.656250
3,2003,10,16,656,1797,"Mediate, Rocco",1,12:10,72,T2,2.5,0.059028
4,2003,10,16,656,1920,"Parry, Craig",1,11:20,71,T25,26.5,0.725694


In [12]:
inds = {num:ind for ind,num in enumerate(pd.unique(data.Player_Number))}

In [15]:
data.insert(5,'Player_Index',[inds[num] for num in data.Player_Number])

In [16]:
rounds = data.groupby(['Tournament_Year','Permanent_Tournament_#','Round_Number','Course_#'])

In [17]:
n_players = len(pd.unique(data.Player_Index))
n_rounds = len(rounds)
n_players,n_rounds

(2594, 2805)

In [18]:
def save_sparse_csc(filename,array):
    np.savez(filename,data = array.data ,indices=array.indices,
             indptr =array.indptr, shape=array.shape )

In [20]:
for round_ind,df in enumerate(rounds):
    df = df[1]
    arr = np.zeros((n_players,n_players))
    arr[np.ix_(pd.unique(df.Player_Index),pd.unique(df.Player_Index))] = np.outer(1/df.Round_Score.values.astype(float),df.Round_Score.values)
    np.fill_diagonal(arr,0)
    arr = csc_matrix(arr)
    save_sparse_csc('./../rounds/%d' % (round_ind),arr)
    if round_ind%300==0:
        print round_ind

0
300
600
900
1200
1500
1800
2100
2400
2700
